In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import module, secret_keys
from model_list import models
import pandas as pd

hf_api_key             = secret_keys.HF_TOKEN                   #<insert your own huggingface token here>
openai_api_key         = secret_keys.OPENAI_API_KEY_TEAM        #<insert your own openai token here>

In [17]:
data = pd.read_csv('hidden_data/CT-Pub-With-Examples-Corrected-allgpteval.csv')

In [18]:
data.shape

(103, 5)

In [19]:
data.head()

,NCTId,gpt4o_zs_gen_matches,gpt4o_ts_gen_matches,llama3_70b_it_zs_gen_matches,llama3_70b_it_ts_gen_matches
0,NCT00000620,"{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ..."
1,NCT00126737,"{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ..."
2,NCT00283686,"{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ..."
3,NCT00329030,"{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ..."
4,NCT00360334,"{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ...","{\n ""matched_features"": [\n [""Age"", ..."


In [20]:
score_df = pd.DataFrame()
score_df['NCTId'] = data['NCTId']

In [21]:
import json 

gzs_sum = {'precision':0, 'recall':0, 'f1':0}
gts_sum = {'precision':0, 'recall':0, 'f1':0}
lzs_sum = {'precision':0, 'recall':0, 'f1':0}
lts_sum = {'precision':0, 'recall':0, 'f1':0}

for index, row in data.iterrows():
    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    #gzs
    gzs_dict = json.loads(row['gpt4o_zs_gen_matches'])
    gzs_matches, gzs_remaining_references, gzs_remaining_candidates = gzs_dict['matched_features'], gzs_dict['remaining_reference_features'], gzs_dict['remaining_candidate_features']
    gzs_score = module.match_to_score(gzs_matches, gzs_remaining_references, gzs_remaining_candidates)
    score_df.at[index, 'gpt4o_zs_gen_scores'] = json.dumps(gzs_score)
    gzs_sum['precision'] += gzs_score['precision']
    gzs_sum['recall'] += gzs_score['recall']
    gzs_sum['f1'] += gzs_score['f1']

    #gts
    gts_dict = json.loads(row['gpt4o_ts_gen_matches'])
    gts_matches, gts_remaining_references, gts_remaining_candidates = gts_dict['matched_features'], gts_dict['remaining_reference_features'], gts_dict['remaining_candidate_features']
    gts_score = module.match_to_score(gts_matches, gts_remaining_references, gts_remaining_candidates)
    score_df.at[index, 'gpt4o_ts_gen_scores'] = json.dumps(gts_score)
    gts_sum['precision'] += gts_score['precision']
    gts_sum['recall'] += gts_score['recall']
    gts_sum['f1'] += gts_score['f1']

    #lzs
    lzs_dict = json.loads(row['llama3_70b_it_zs_gen_matches'])
    lzs_matches, lzs_remaining_references, lzs_remaining_candidates = lzs_dict['matched_features'], lzs_dict['remaining_reference_features'], lzs_dict['remaining_candidate_features']
    lzs_score = module.match_to_score(lzs_matches, lzs_remaining_references, lzs_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_zs_gen_scores'] = json.dumps(lzs_score)
    lzs_sum['precision'] += lzs_score['precision']
    lzs_sum['recall'] += lzs_score['recall']
    lzs_sum['f1'] += lzs_score['f1']

    #lts 
    lts_dict = json.loads(row['llama3_70b_it_ts_gen_matches'])
    lts_matches, lts_remaining_references, lts_remaining_candidates = lts_dict['matched_features'], lts_dict['remaining_reference_features'], lts_dict['remaining_candidate_features']
    lts_score = module.match_to_score(lts_matches, lts_remaining_references, lts_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_ts_gen_scores'] = json.dumps(lts_score)
    lts_sum['precision'] += lts_score['precision']
    lts_sum['recall'] += lts_score['recall']
    lts_sum['f1'] += lts_score['f1']

    #break



In [29]:
score_df.head()

,NCTId,gpt4o_zs_gen_scores,gpt4o_ts_gen_scores,llama3_70b_it_zs_gen_scores,llama3_70b_it_ts_gen_scores
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00126737,"{""precision"": 0.5555555555555556, ""recall"": 0....","{""precision"": 0.35, ""recall"": 0.58333333333333...","{""precision"": 0.36363636363636365, ""recall"": 0...","{""precision"": 0.4166666666666667, ""recall"": 0...."
2,NCT00283686,"{""precision"": 0.8571428571428571, ""recall"": 0....","{""precision"": 0.4, ""recall"": 0.2, ""f1"": 0.2666...","{""precision"": 0.375, ""recall"": 0.1621621621621...","{""precision"": 0.23809523809523808, ""recall"": 0..."
3,NCT00329030,"{""precision"": 0.17391304347826086, ""recall"": 0...","{""precision"": 0.3125, ""recall"": 0.625, ""f1"": 0...","{""precision"": 0.3, ""recall"": 0.75, ""f1"": 0.428...","{""precision"": 0.3125, ""recall"": 0.625, ""f1"": 0..."
4,NCT00360334,"{""precision"": 0.4666666666666667, ""recall"": 0....","{""precision"": 0.47368421052631576, ""recall"": 1...","{""precision"": 0.5333333333333333, ""recall"": 0....","{""precision"": 0.35, ""recall"": 0.77777777777777..."


In [30]:
score_df.to_csv('hidden_data/CT-Pub-With-Examples-Corrected-gptmetric.csv', index=False)

In [28]:
print(f"GPT-4o Zero Shot Scores: Precision={gzs_sum['precision']/100} Recall={gzs_sum['recall']/100} F1={gzs_sum['f1']/100}")
print(f"GPT-4o Three Shot Scores: Precision={gts_sum['precision']/100} Recall={gts_sum['recall']/100} F1={gts_sum['f1']/100}")
print(f"Llama-3 70B Instruct Zero Shot Scores: Precision={lzs_sum['precision']/100} Recall={lzs_sum['recall']/100} F1={lzs_sum['f1']/100}")
print(f"Llama-3 70B Instruct Three Shot Scores: Precision={lts_sum['precision']/100} Recall={lts_sum['recall']/100} F1={lts_sum['f1']/100}")

GPT-4o Zero Shot Scores: Precision=0.4058158031331031 Recall=0.5253822091625672 F1=0.43123220496938564
GPT-4o Three Shot Scores: Precision=0.40578342345214774 Recall=0.58334349386987 F1=0.4558358458155929
Llama-3 70B Instruct Zero Shot Scores: Precision=0.43800161220788164 Recall=0.5686781285175699 F1=0.4738921785330177
Llama-3 70B Instruct Three Shot Scores: Precision=0.42949291888022545 Recall=0.5787174189522326 F1=0.470271674345745
